# Multiple Linear Regression

### Formula : y = b0 + b1* x1 + b2* x2 + b3* x3 


Assumptions of a Linear Regression (Before heading to build a Linear Regression Model, we must make the following points TRUE):
- Linearity
- Homoscendasticity
- Multivariate normality
- Independence of errors
- Lack of multicollinearity

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as tts


import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("50_Startups.csv")
print(df.shape)
print(df.describe())
print("Number of Nan \n", df.isnull().sum())

(50, 5)
           R&D Spend  Administration  Marketing Spend         Profit
count      50.000000       50.000000        50.000000      50.000000
mean    73721.615600   121344.639600    211025.097800  112012.639200
std     45902.256482    28017.802755    122290.310726   40306.180338
min         0.000000    51283.140000         0.000000   14681.400000
25%     39936.370000   103730.875000    129300.132500   90138.902500
50%     73051.080000   122699.795000    212716.240000  107978.190000
75%    101602.800000   144842.180000    299469.085000  139765.977500
max    165349.200000   182645.560000    471784.100000  192261.830000
Number of Nan 
 R&D Spend          0
Administration     0
Marketing Spend    0
State              0
Profit             0
dtype: int64


In [4]:
df.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


### Dummy Variables

Dummy Variables are created to deal with the categorical variables, as we cant fit categorical variables in a regression problem.

Use- pd.get_dummies() to get one hot encoding of categorical variables. 

### Dummy Variable Trap

We never include all the dummy variables to fit the model:-
- suppose we have two classes, so one dummy variable(one-hot encoded) would give us the information of both the class (D2 = 1 - D1) 
- In such cases, where two variables are related to each other, like one could predict other. 
    - _**The Phenomenon where one or several independent variables in a linear regression predict another is called Multicollinearity**._

As a result of this effect, the model cannot distinguish between the effects of such variables. 

**Thus, ALWAYS omit one dummy variable**

In [6]:
#Dummy Variables (Dealing with categorical variables in Regression problem)
pd.get_dummies(df['State'],drop_first=True)    #dummy variables work as light switches

,Florida,New York
0,0,1
1,0,0
2,1,0
3,0,1
4,1,0
5,0,1
6,0,0
7,1,0
8,0,1
9,0,0


In [10]:
# df.corr()        #used for getting multicolinearity

## P Value (alpha)

P value is a statistical measure that helps scientists determine whether or not their hypotheses are correct. P values are used to determine whether the results of their experiment are within the normal range of values for the events being observed. Usually, if the P value of a data set is below a certain pre-determined amount (like, for instance, 0.05), scientists will reject the "null hypothesis" of their experiment - in other words, they'll rule out the hypothesis that the variables of their experiment had no meaningful effect on the results. Today, p values are usually found on a reference table by first calculating a chi square value.  (https://www.wikihow.com/Calculate-P-Value)

**Hence we _DISCARD_ the feature with _P-value>0.05_**

Read about **Hypothesis Testing** (https://towardsdatascience.com/hypothesis-testing-in-machine-learning-using-python-a0dc89e169ce)

### Building A Model (in case of multiple features)

- we so this to avoid large number of features, and settle down with sensible features those actually matter to use while building/training a Model

#### 5 Methids of building models:
- All-in
- Backward Elimination
- Forward Selection
- Bidirectional Elimination
- Score Comparison

Stepwise Regression (same as point numbers 2,3,4)

- **1) All-in**
    - Here we take all features.
- **2) Backward Elimination**
    - Select a significance level to stay in the model (eg. SL=0.05)
    - Fit the full model with all posible predictors (all features)
    - Consider the predictor with the **highest P-value**. _**If P>SL, go to next step, otherwise go to FIN(model is ready)**_
    - Remove that predictor (feature/variable)
    - Fit model without this variable 
    - Go back to step 3, until the condition of highest P-value>SL becomes False
- **3) Forward Selection**
    - Select a significance level to enter the model (eg. SL=0.05)
    - Fit all simple regression models y ~ x.(one model for each feature). **Select the one with the lowest P-value.**
    - Keep this variable and fit all possible models with one extra predictor added to the one(s) you already have. (here we add one variable, the one we just selected with lowest p-value, to all other models)
    - Consider the predictor with the _lowest_ P-value. If P < SL, go to above step, otherwise go to FIN
    - FIN: Keep the previous model, just befor getting the model with P>SL(we will discard this) and take previous model.
- **4)Bidirectional Elimination**
    - Select a significance level to enter and to stay in the model. (eg: SLENTER=0.05, SLSTAY=0.05)
    - Perform the next step of _Forward Selection_ (new variable must have P<SLENTER to enter)
    - Perform ALL steps of _Backward Elimination_ (old variables must have P<SLSTAY to stay)
    - Loop up(iterate) with step 2-3, until no new varuables van enter and no old variables can exit. Model is Ready.
- **5) All Possible Models/ Score Comparison**
    - Select a criterion of goodness of fit(eg. Akaike Criterion)
    - Contruct ALL POSSIBLE Regression models: **(2^N)-1** total combinations
    - Select the one with the best criterion
    - FIN: Model is ready.  (**This method sounds easy, but shouldn't be used as it is cost consuming and resource consuming, not affordable for more than 5-10 features)
    
    
_NOTE_: The **Backward Elimination** method is **Fastest** of all the model building methods. We will always prioratise this method.
 

In [56]:
X = df[['R&D Spend', 'Administration', 'Marketing Spend', 'State']]       #NOTE: input independent variable should be a 2D array, not 1D array
y = df['Profit']

In [57]:
#One-Hot Encoding for 'State' column
X = pd.concat([X,pd.get_dummies(X['State'],drop_first=True)],axis=1)    #dropped 1st class to avoid Dummy variable trap
X.head()

,R&D Spend,Administration,Marketing Spend,State,Florida,New York
0,165349.20,136897.80,471784.10,New York,0,1
1,162597.70,151377.59,443898.53,California,0,0
2,153441.51,101145.55,407934.54,Florida,1,0
3,144372.41,118671.85,383199.62,New York,0,1
4,142107.34,91391.77,366168.42,Florida,1,0


In [58]:
X= X.drop("State", axis=1)
X.head()

,R&D Spend,Administration,Marketing Spend,Florida,New York
0,165349.20,136897.80,471784.10,0,1
1,162597.70,151377.59,443898.53,0,0
2,153441.51,101145.55,407934.54,1,0
3,144372.41,118671.85,383199.62,0,1
4,142107.34,91391.77,366168.42,1,0


In [59]:
X_train, X_test, y_train, y_test = tts(X,y,test_size=0.2, random_state=42)

In [60]:
#Fitting Multiple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [61]:
y_pred = regressor.predict(X_test)
y_pred, y_test     #this model is quit a good linear fit, as y_pred is comparable to y_test 

(array([126362.87908255,  84608.45383634,  99677.49425147,  46357.46068582,
        128750.48288504,  50912.4174188 , 109741.35032702, 100643.24281647,
         97599.27574594, 113097.42524432]), 13    134307.35
 39     81005.76
 30     99937.59
 45     64926.08
 17    125370.37
 48     35673.41
 26    105733.54
 25    107404.34
 32     97427.84
 19    122776.86
 Name: Profit, dtype: float64)

In [66]:
#Building the optimal model using Backward Elimination
import statsmodels.api as sm


# #add a feature with all ones in it to satisfy the need of constant term(b0) in multiple linear regression equation for backward elimination's statsmodel library
# X['All_ones'] = 1
# X = X[['All_ones','R&D Spend', 'Administration', 'Marketing Spend', 'Florida', 'New York']]   #added the all 1s columns at the beginning

#the above work can also be done in this way 
X = sm.add_constant(X)      #this will automatically append a column at the beginning
X.head()

,const,R&D Spend,Administration,Marketing Spend,Florida,New York
0,1.0,165349.20,136897.80,471784.10,0,1
1,1.0,162597.70,151377.59,443898.53,0,0
2,1.0,153441.51,101145.55,407934.54,1,0
3,1.0,144372.41,118671.85,383199.62,0,1
4,1.0,142107.34,91391.77,366168.42,1,0


In [70]:
#X_optimal will contain only the feature which have high impact on the dependent variable
X_optimal = X         #initializing with all the feature, 
SL = 0.05             #step 1 of Backward Elimination

regressor_OLS = sm.OLS(endog=y, exog=X_optimal).fit()     #step 2
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Sun, 23 Feb 2020   Prob (F-statistic):           1.34e-27
Time:                        18:27:40   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            5.013e+04   6884.820      7.281      0.000    3.62e+04     6.4e+04
R&D Spend           0.8060      0.046     17.369      0.000       0.712       0.900
Administration     -0.0270      0.052     -0.517      0.608      -0.132       0.078
Marketing Spend     0.0270      0.017      1.574      0.123      -0.008       0.062
Florida           198.7888   3371.007      0.059      0.953   -6595.030    6992.607
New York          -41.8870   3256.039     -0.013      0.990   -6604.003    6520.229
==============================================================================
Omnibus:                       14.782   Durbin-Watson:                   1.283
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.266
Skew:                          -0.948   Prob(JB):                     2.41e-05
Kurtosis:                       5.572   Cond. No.                     1.45e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.45e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [72]:
#step 3,4, from above summary select the variable with highest P-value which is variable "New York" and this is greater than 0.05, hence we will remove this feature.
X_optimal = X[['const', 'R&D Spend', 'Administration', 'Marketing Spend', 'Florida']]         #initializing without "New York"

regressor_OLS = sm.OLS(endog=y, exog=X_optimal).fit()     #back to step 2
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     217.2
Date:                Sun, 23 Feb 2020   Prob (F-statistic):           8.49e-29
Time:                        18:33:07   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1061.
Df Residuals:                      45   BIC:                             1070.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            5.011e+04   6647.870      7.537      0.000    3.67e+04    6.35e+04
R&D Spend           0.8060      0.046     17.606      0.000       0.714       0.898
Administration     -0.0270      0.052     -0.523      0.604      -0.131       0.077
Marketing Spend     0.0270      0.017      1.592      0.118      -0.007       0.061
Florida           220.1585   2900.536      0.076      0.940   -5621.821    6062.138
==============================================================================
Omnibus:                       14.758   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.172
Skew:                          -0.948   Prob(JB):                     2.53e-05
Kurtosis:                       5.563   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [73]:
#similar to previous statement we will now drop "Florida" which has P-value = 0.94 (>0.05)
X_optimal = X[['const', 'R&D Spend', 'Administration', 'Marketing Spend']]         #initializing without "Florida"

regressor_OLS = sm.OLS(endog=y, exog=X_optimal).fit()     #back to step 2
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     296.0
Date:                Sun, 23 Feb 2020   Prob (F-statistic):           4.53e-30
Time:                        18:34:36   Log-Likelihood:                -525.39
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      46   BIC:                             1066.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            5.012e+04   6572.353      7.626      0.000    3.69e+04    6.34e+04
R&D Spend           0.8057      0.045     17.846      0.000       0.715       0.897
Administration     -0.0268      0.051     -0.526      0.602      -0.130       0.076
Marketing Spend     0.0272      0.016      1.655      0.105      -0.006       0.060
==============================================================================
Omnibus:                       14.838   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.442
Skew:                          -0.949   Prob(JB):                     2.21e-05
Kurtosis:                       5.586   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [75]:
#this time we will drop "Administration" which has P-value = 0.6 (>0.05)
X_optimal = X[['const', 'R&D Spend', 'Marketing Spend']]         #initializing without "Administration"

regressor_OLS = sm.OLS(endog=y, exog=X_optimal).fit()     #back to step 2
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     450.8
Date:                Sun, 23 Feb 2020   Prob (F-statistic):           2.16e-31
Time:                        18:36:13   Log-Likelihood:                -525.54
No. Observations:                  50   AIC:                             1057.
Df Residuals:                      47   BIC:                             1063.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            4.698e+04   2689.933     17.464      0.000    4.16e+04    5.24e+04
R&D Spend           0.7966      0.041     19.266      0.000       0.713       0.880
Marketing Spend     0.0299      0.016      1.927      0.060      -0.001       0.061
==============================================================================
Omnibus:                       14.677   Durbin-Watson:                   1.257
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.161
Skew:                          -0.939   Prob(JB):                     2.54e-05
Kurtosis:                       5.575   Cond. No.                     5.32e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.32e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [76]:
#now drop "Marketing Spend" with P-value = 0.06 (>0.05). Ideally we will keep this feature next time, where we will also consider R-squared values.
X_optimal = X[['const', 'R&D Spend']]         #initializing without "Marketing Spend"

regressor_OLS = sm.OLS(endog=y, exog=X_optimal).fit()     #back to step 2
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     849.8
Date:                Sun, 23 Feb 2020   Prob (F-statistic):           3.50e-32
Time:                        18:37:25   Log-Likelihood:                -527.44
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      48   BIC:                             1063.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.903e+04   2537.897     19.320      0.000    4.39e+04    5.41e+04
R&D Spend      0.8543      0.029     29.151      0.000       0.795       0.913
==============================================================================
Omnibus:                       13.727   Durbin-Watson:                   1.116
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               18.536
Skew:                          -0.911   Prob(JB):                     9.44e-05
Kurtosis:                       5.361   Cond. No.                     1.65e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Now we have no P-value > 0.05 hence we will settle down on this X_optimal

## Automatic Backward Elimination

In [112]:
import statsmodels.api as sm

def backwardElimination(x,sl):
    x = sm.add_constant(x)
    numVars = len(x.columns)
    for i in range(numVars):
        regressor_OLS = sm.OLS(y,x).fit()
        maxVar = max(regressor_OLS.pvalues)
        print(maxVar,np.argmax(regressor_OLS.pvalues))
        #print(regressor_OLS.summary())
        if maxVar > sl:
            x = x.drop([np.argmax(regressor_OLS.pvalues)], axis=1)
            print(x.columns)
        else:
            break
    return x

In [114]:
sl = 0.05
X_opt = X
X_final = backwardElimination(X_opt,sl)
# X_final

0.9897941241607553 New York
Index(['const', 'R&D Spend', 'Administration', 'Marketing Spend', 'Florida'], dtype='object')
0.9398329772576753 Florida
Index(['const', 'R&D Spend', 'Administration', 'Marketing Spend'], dtype='object')
0.6017551078497458 Administration
Index(['const', 'R&D Spend', 'Marketing Spend'], dtype='object')
0.06003039719113059 Marketing Spend
Index(['const', 'R&D Spend'], dtype='object')
2.7826969229654186e-24 const


In [81]:
len(X.columns)

6

In [115]:
max(regressor_OLS.pvalues), type(regressor_OLS.pvalues)

(2.7826969229654186e-24, pandas.core.series.Series)

In [116]:
(regressor_OLS.pvalues)

const        2.782697e-24
R&D Spend    3.500322e-32
dtype: float64

In [107]:
np.argmax(regressor_OLS.pvalues)

'const'

In [16]:
#Checking multicolinearity in our data set

,R&D Spend,Administration,Marketing Spend
R&D Spend,1.000000,0.241955,0.724248
Administration,0.241955,1.000000,-0.032154
Marketing Spend,0.724248,-0.032154,1.000000


TypeError: 'module' object is not callable